In [1]:
import numpy as np

In [2]:
!pip install kafka-python

In [3]:
!pip install pyspark

In [4]:
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import rand, randn, mean, stddev, to_date, col, when
from datetime import datetime, timedelta
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import json
import random
import time
import os
import plotly.graph_objs as go
from plotly.subplots import make_subplots

C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
print("JAVA_HOME:", os.getenv("JAVA_HOME"))

if not os.getenv("JAVA_HOME"):
    java_home = "C:\\Program Files\\Java\\jdk-23"  # Adjust this path to your Java installation
    os.environ["JAVA_HOME"] = java_home
    print("JAVA_HOME set to:", os.getenv("JAVA_HOME"))

        
# Now append the Java bin directory to PATH
os.environ["PATH"] = os.getenv("JAVA_HOME") + "\\bin;" + os.getenv("PATH")
print("Updated PATH:", os.getenv("PATH"))


JAVA_HOME: None
JAVA_HOME set to: C:\Program Files\Java\jdk-23
Updated PATH: C:\Program Files\Java\jdk-23\bin;C:\Users\hp\anaconda3;C:\Users\hp\anaconda3\Library\mingw-w64\bin;C:\Users\hp\anaconda3\Library\usr\bin;C:\Users\hp\anaconda3\Library\bin;C:\Users\hp\anaconda3\Scripts;c:\Users\hp\AppData\Local\Programs\cursor\resources\app\bin;C:\windows\system32;C:\Users\hp\AppData\Roaming\Python\Python311\Scripts;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0\;C:\windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\MinGW\mingw32\bin;C:\Program Files\nodejs\;C:\Program Files\Git\cmd;C:\Program Files\HP\HP One Agent;C:\Program Files\MySQL\MySQL Shell 8.0\bin\;C:\Users\hp\AppData\Local\Microsoft\WindowsApps;C:\Program Files\Python311;C:\Program Files\Python311\Script

In [6]:
!java -version

java version "23" 2024-09-17
Java(TM) SE Runtime Environment (build 23+37-2369)
Java HotSpot(TM) 64-Bit Server VM (build 23+37-2369, mixed mode, sharing)


In [8]:
!pip uninstall pyspark


^C


In [9]:
!pip install pyspark

In [12]:
spark = SparkSession.getActiveSession()
print(spark)
if spark is not None:
    spark.stop()

if SparkContext._active_spark_context is not None:
    SparkContext._active_spark_context.stop()

print(SparkContext._active_spark_context)

app_name = "IoT_Home_Appliances"
spark = SparkSession.builder.appName(app_name).getOrCreate()
spark

None
None


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [13]:
!echo %JAVA_HOME%  # Windows


C:\Program Files\Java\jdk-23  # Windows


In [15]:
spark = SparkSession.builder.appName("Test App").getOrCreate()
spark

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [ ]:
df = spark.read.csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv', header=True, inferSchema=True)
df.show()

In [ ]:
df = df.drop("meanpressure","wind_speed")
df.show()

In [ ]:
df.count()

In [ ]:
stats = df.select(
    mean(df.humidity).alias("mean_humidity"),
    stddev(df.humidity).alias("stddev_humidity")
).collect()

mean_humidity = stats[0]['mean_humidity']
stddev_humidity = stats[0]['stddev_humidity']

In [ ]:
mean_humidity, stddev_humidity

In [ ]:
# Convert the 'date' column to actual date type (assuming date format is 'YYYY-MM-DD')
df = df.withColumn('date', to_date(df.date, 'yyyy-MM-dd'))

# Prepare features using VectorAssembler
assembler = VectorAssembler(inputCols=["humidity"], outputCol="features")
df_assembled = assembler.transform(df)

# Assume 'meantemp' is the label column
df_assembled = df_assembled.select('date', 'features', 'meantemp')

In [ ]:
# Split the data into training and testing (assuming you use all data for training here)
train_data = df_assembled  # As this is a simulation, we'll assume all the data is for training

# Apply Linear Regression
linear_reg = LinearRegression(featuresCol='features', labelCol='meantemp')
model = linear_reg.fit(train_data)


In [ ]:
df_assembled.show()

In [ ]:
start_date = "2017-01-02"
end_date = "2017-05-31"
date_index = pd.date_range(start_date, end_date, freq='D')

num_days = len(date_index)
random_humidities = np.random.normal(loc=mean_humidity, scale=stddev_humidity, size=num_days)

# Ensure humidity values are within a realistic range (e.g., 0 to 100)
random_humidities = np.clip(random_humidities, 0, 100)

# Create a Pandas DataFrame with the date range and generated humidity values
pandas_df = pd.DataFrame({
    'date': date_index,
    'humidity': random_humidities
})

pandas_df

In [ ]:
new_data = spark.createDataFrame(pandas_df)

# Convert 'date' to the proper date type
new_data = new_data.withColumn('date', to_date(new_data.date))

# Prepare the features using VectorAssembler
assembler = VectorAssembler(inputCols=["humidity"], outputCol="features")
new_data = assembler.transform(new_data)

# Display the prepared DataFrame for verification
new_data.show()

In [ ]:
# Predict using the model
predicted_data = model.transform(new_data)
generated_data = predicted_data.select('date', 'humidity', 'prediction')
generated_data.show()

In [ ]:
df.show()

In [ ]:
generated_data = generated_data.withColumnRenamed("prediction","meantemp")

In [ ]:
generated_data.show()

In [ ]:
final_df = df.unionByName(generated_data)
final_df.show()

In [ ]:
baseline_temp = 20.0
baseline_humidity = 50.0

In [ ]:
final_df = final_df.withColumn("price", 
                   when(col("meantemp") > baseline_temp, (col("meantemp") - baseline_temp) * 1) \
                   .otherwise(0) - \
                   when(col("humidity") > baseline_humidity, (col("humidity") - baseline_humidity) * 0.5) \
                   .otherwise(0)
                  )

# Adjust price to have a base price of $50
final_df = final_df.withColumn("price", col("price") + 50)

final_df.show()

In [ ]:
pd_df = final_df.toPandas()
pd_df

In [ ]:
pd_df['date'] = pd.to_datetime(pd_df['date'])

In [ ]:
# Create a figure with subplots
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02,  # Vertical spacing between plots
                    subplot_titles=("Mean Temperature", "Humidity", "Price"))

# Temperature Plot
fig.add_trace(
    go.Scatter(x=pd_df['date'], y=pd_df['meantemp'], name='Mean Temperature', mode='lines', line=dict(color='red')),
    row=1, col=1
)

# Humidity Plot
fig.add_trace(
    go.Scatter(x=pd_df['date'], y=pd_df['humidity'], name='Humidity', mode='lines', line=dict(color='blue')),
    row=2, col=1
)

# Price Plot
fig.add_trace(
    go.Scatter(x=pd_df['date'], y=pd_df['price'], name='Price', mode='lines', line=dict(color='green')),
    row=3, col=1
)

# Update x-axis and y-axis labels
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_yaxes(title_text="Humidity (%)", row=2, col=1)
fig.update_yaxes(title_text="Price ($)", row=3, col=1)

# Update layout and plot size
fig.update_layout(height=900, width=700, title_text="Data Over Time", showlegend=False)

# Show plot
fig.show()

In [ ]:
final_df.write.csv('./final_data.csv', header=True, mode="overwrite")

In [ ]:
# List files in the current working directory
files = os.listdir('.')
print(files)

In [ ]:
from kafka import KafkaProducer
import csv

producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Function to send records to a Kafka topic
def send_record(topic_name, value):
    producer.send(topic_name, value.encode('utf-8'))

# Assuming CSV file format: date,meantemp,humidity,price
with open('data.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header
    for row in reader:
        send_record('temperature', f"{row[0]},{row[1]}")
        send_record('humidity', f"{row[0]},{row[2]}")
        send_record('price', f"{row[0]},{row[3]}")

producer.flush()


In [ ]:
from kafka import KafkaConsumer

# Initialize consumer
consumer = KafkaConsumer(
    'temperature',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest'
)

# Read messages
for message in consumer:
    print(f"Received message: {message.value.decode('utf-8')}")
